In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

print(torch.cuda.get_device_name())
print(torch.__version__, torch.cuda.is_available())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

NVIDIA RTX A2000 8GB Laptop GPU
2.0.0+cu118 True
cuda:0


In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [4]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, output_seq_length, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.output_seq_length = output_seq_length
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=self.num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        #入力のsequence長さを出力のsequence長さに合わせるため拡張する
        new_size = (x.size(0), self.output_seq_length, input_size)
        x = x.expand(*new_size)
        
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        output, _ = self.lstm(x, (h0, c0))
        output = self.fc(output)

        return output

In [15]:
#MLP学習済みモデルをセットする

# ハイパーパラメータの設定
input_size = 6
hidden_size = 2000
output_size = 56

#MLP
MLP_env1 = MLP(input_size, hidden_size, output_size).to(device)
MLP_env1.load_state_dict(torch.load('../saved_models/MLP_env1.pth'))

MLP_env1_extract = MLP(input_size, hidden_size, output_size).to(device)
MLP_env1_extract.load_state_dict(torch.load('../saved_models/MLP_env1_first_quantile.pth'))

<All keys matched successfully>

In [16]:
#LSTM学習済みモデルをセットする

# ハイパーパラメータの設定
input_size = 6
hidden_size = 128
output_size = 2
output_seq_length = 28
num_layers = 1

#LSTM
LSTM_env1 = LSTM(input_size, hidden_size, output_size, output_seq_length, num_layers).to(device)
LSTM_env1.load_state_dict(torch.load('../saved_models/LSTM_env1.pth'))

LSTM_env1_extract = LSTM(input_size, hidden_size, output_size, output_seq_length, num_layers).to(device)
LSTM_env1_extract.load_state_dict(torch.load('../saved_models/LSTM_env1_first_quantile.pth'))

<All keys matched successfully>

In [17]:
class TestDataset(Dataset):
    def __init__(self, test_data):
        self.test_data = test_data

    def __len__(self):
        return len(self.test_data)

    def __getitem__(self, index):
        input = torch.tensor(self.test_data[index], dtype=torch.float32)
        output = 0
        return input, output

In [18]:
#検証
#モデルで予測したpathがcollision freeである割合を計算する
import random
import utils
import plot
import matplotlib.pyplot as plt

#検証回数
max_iter = 1000

counter = {MLP_env1:0, MLP_env1_extract:0, LSTM_env1:0, LSTM_env1_extract:0}

for iteration in range(max_iter):
    
    #初期状態・終端状態のサンプリング
    x_start = (random.uniform(-2, 4), random.uniform(-3, 3))  # Starting node
    x_goal = (random.uniform(26, 32), random.uniform(-3, 3))  # Goal node
    theta_start = random.uniform(-np.pi/2, np.pi/2)
    theta_goal = random.uniform(-np.pi/2, np.pi/2)

    #test dataのset
    test_data = np.array([[x_start[0], x_goal[0], x_start[1], x_goal[1], theta_start, theta_goal]])
    test_data = TestDataset(test_data)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)
    
    ####
    #MLP_env1
    ####
    MLP_env1.eval()
    with torch.no_grad():
        Iter = test_loader.__iter__()
        input_data, _ = next(Iter)
        input_data = input_data.to(device)
        output = MLP_env1(input_data)
        output = output.view(2, -1)
        
    output = output.cpu().numpy()
    test_x_path = np.append(output[0], x_goal[0])
    test_x_path = np.append(x_start[0], test_x_path)
    test_y_path = np.append(output[1], x_goal[1])
    test_y_path = np.append(x_start[1], test_y_path)
    
    if utils.is_collision_free(test_x_path, test_y_path) == True:
        counter[MLP_env1] = counter[MLP_env1] + 1
    else:
        pass
    
    ####
    #MLP_env1_extract
    ####
    MLP_env1_extract.eval()
    with torch.no_grad():
        Iter = test_loader.__iter__()
        input_data, _ = next(Iter)
        input_data = input_data.to(device)
        output = MLP_env1_extract(input_data)
        output = output.view(2, -1)
        
    output = output.cpu().numpy()
    test_x_path = np.append(output[0], x_goal[0])
    test_x_path = np.append(x_start[0], test_x_path)
    test_y_path = np.append(output[1], x_goal[1])
    test_y_path = np.append(x_start[1], test_y_path)
    
    if utils.is_collision_free(test_x_path, test_y_path) == True:
        counter[MLP_env1_extract] = counter[MLP_env1_extract] + 1
    else:
        pass 
    
    ####
    #LSTM_env1
    ####
    LSTM_env1.eval()
    with torch.no_grad():
        Iter = test_loader.__iter__()
        input_data, _ = next(Iter)
        input_data = input_data.to(device)
        output = LSTM_env1(input_data)
        
    output = output.cpu().numpy()    
    test_x_path = np.array([])
    test_y_path = np.array([])
    for i in range(output.shape[1]):
        test_x_path = np.append(test_x_path, output[0, i, 0])
        test_y_path = np.append(test_y_path, output[0, i, 1])
        
    test_x_path = np.append(test_x_path, x_goal[0])
    test_x_path = np.append(x_start[0], test_x_path)
    test_y_path = np.append(test_y_path, x_goal[1])
    test_y_path = np.append(x_start[1], test_y_path)
    
    if utils.is_collision_free(test_x_path, test_y_path) == True:
        counter[LSTM_env1] = counter[LSTM_env1] + 1
    else:
        pass 
    
    ####
    #LSTM_env1_extract
    ####
    LSTM_env1_extract.eval()
    with torch.no_grad():
        Iter = test_loader.__iter__()
        input_data, _ = next(Iter)
        input_data = input_data.to(device)
        output = LSTM_env1_extract(input_data)
        
    output = output.cpu().numpy()    
    test_x_path = np.array([])
    test_y_path = np.array([])
    for i in range(output.shape[1]):
        test_x_path = np.append(test_x_path, output[0, i, 0])
        test_y_path = np.append(test_y_path, output[0, i, 1])
        
    test_x_path = np.append(test_x_path, x_goal[0])
    test_x_path = np.append(x_start[0], test_x_path)
    test_y_path = np.append(test_y_path, x_goal[1])
    test_y_path = np.append(x_start[1], test_y_path)
    
    if utils.is_collision_free(test_x_path, test_y_path) == True:
        counter[LSTM_env1_extract] = counter[LSTM_env1_extract] + 1
    else:
        pass 


In [19]:
for v in counter.values():
    print('{}/1000'.format(v))

16/1000
596/1000
7/1000
709/1000


In [20]:
18/1000
347/1000
9/1000
386/1000

9/1000
362/1000
10/1000
405/1000

16/1000
596/1000
7/1000
709/1000

0.709